In [1]:

import os
import sys
sys.path.append('/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/')
import argparse
import numpy as np
import nibabel as nib
import scipy.io as sio
import subprocess
from scipy.stats import zscore
from nibabel.nicom import dicomreaders
import pydicom as dicom  # type: ignore
import time
from glob import glob
import shutil
from nilearn.image import new_img_like
import joblib
import rtCommon.utils as utils
from rtCommon.utils import loadConfigFile
import pickle5 as pickle
# import and set up environment
import sys
from subprocess import call
import nibabel as nib
import pydicom as dicom
import numpy as np
import time
import os
from glob import glob
import shutil
import pandas as pd
# from import convertDicomFileToNifti
from rtCommon.imageHandling import convertDicomImgToNifti, readDicomFromFile
from rtCommon.cfg_loading import mkdir,cfg_loading

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
# from rtCommon.fileClient import FileInterface
# import rtCommon.projectUtils as projUtils
# from rtCommon.imageHandling import readRetryDicomFromFileInterface, getDicomFileName, convertDicomImgToNifti


argParser = argparse.ArgumentParser()
argParser.add_argument('--config', '-c', default='sub001.ses1.toml', type=str, help='experiment file (.json or .toml)')
argParser.add_argument('--skipPre', '-s', default=0, type=int, help='skip preprocess or not')
argParser.add_argument('--skipGreedy', '-g', default=0, type=int, help='skip greedy or not')
args = argParser.parse_args("")
from rtCommon.cfg_loading import mkdir,cfg_loading
# config="sub001.ses2.toml"
cfg = cfg_loading(args.config)



/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  # Remove the CWD from sys.path while we load stuff.


conda env=/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud
toml filename=/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/projects/rtSynth_rt/conf/sub001.ses1.toml


In [186]:
# def minimalClass(cfg):
'''
purpose: 
    train offline models

steps:
    load preprocessed and aligned behavior and brain data 
    select data with the wanted pattern like AB AC AD BC BD CD 
    train correspondng classifier and save the classifier performance and the classifiers themselves.

'''

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import joblib
import nibabel as nib
import itertools
from sklearn.linear_model import LogisticRegression

def gaussian(x, mu, sig):
    # mu and sig is determined before each neurofeedback session using 2 recognition runs.
    return round(1+18*(1 - np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))))) # map from (0,1) -> [1,19]

def jitter(size,const=0):
    jit = np.random.normal(0+const, 0.05, size)
    X = np.zeros((size))
    X = X + jit
    return X

def other(target):
    other_objs = [i for i in ['bed', 'bench', 'chair', 'table'] if i not in target]
    return other_objs

def red_vox(n_vox, prop=0.1):
    return int(np.ceil(n_vox * prop))

def get_inds(X, Y, pair, testRun=None):
    
    inds = {}
    
    # return relative indices
    if testRun:
        trainIX = Y.index[(Y['label'].isin(pair)) & (Y['run_num'] != int(testRun))]
    else:
        trainIX = Y.index[(Y['label'].isin(pair))]

    # pull training and test data
    trainX = X[trainIX]
    trainY = Y.iloc[trainIX].label
    
    # Main classifier on 5 runs, testing on 6th
    clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
                                multi_class='multinomial').fit(trainX, trainY)
    B = clf.coef_[0]  # pull betas

    # retrieve only the first object, then only the second object
    if testRun:
        obj1IX = Y.index[(Y['label'] == pair[0]) & (Y['run_num'] != int(testRun))]
        obj2IX = Y.index[(Y['label'] == pair[1]) & (Y['run_num'] != int(testRun))]
    else:
        obj1IX = Y.index[(Y['label'] == pair[0])]
        obj2IX = Y.index[(Y['label'] == pair[1])]

    # Get the average of the first object, then the second object
    obj1X = np.mean(X[obj1IX], 0)
    obj2X = np.mean(X[obj2IX], 0)

    # Build the importance map
    mult1X = obj1X * B
    mult2X = obj2X * B

    # Sort these so that they are from least to most important for a given category.
    sortmult1X = mult1X.argsort()[::-1]
    sortmult2X = mult2X.argsort()

    # add to a dictionary for later use
    inds[clf.classes_[0]] = sortmult1X
    inds[clf.classes_[1]] = sortmult2X
                
    return inds

if 'milgram' in os.getcwd():
    main_dir='/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/'
else:
    main_dir='/Volumes/GoogleDrive/My Drive/Turk_Browne_Lab/rtcloud_kp/'

working_dir=main_dir
os.chdir(working_dir)

'''
if you read runRecording for current session and found that there are only 4 runs in the current session, 
you read the runRecording for previous session and fetch the last 4 recognition runs from previous session
'''
runRecording = pd.read_csv(f"{cfg.recognition_dir}../runRecording.csv")
actualRuns = list(runRecording['run'].iloc[list(np.where(1==1*(runRecording['type']=='recognition'))[0])]) # can be [1,2,3,4,5,6,7,8] or [1,2,4,5]
if len(actualRuns) < 8:
    runRecording_preDay = pd.read_csv(f"{cfg.subjects_dir}{cfg.subjectName}/ses{cfg.session-1}/recognition/../runRecording.csv")
    actualRuns_preDay = list(runRecording_preDay['run'].iloc[list(np.where(1==1*(runRecording_preDay['type']=='recognition'))[0])])[-(8-len(actualRuns)):] # might be [5,6,7,8]
else: 
    actualRuns_preDay = []

# assert len(actualRuns_preDay)+len(actualRuns)==8 
if len(actualRuns_preDay)+len(actualRuns)<8:
    runRecording_prepreDay = pd.read_csv(f"{cfg.subjects_dir}{cfg.subjectName}/ses{cfg.session-2}/recognition/../runRecording.csv")
    actualRuns_prepreDay = list(runRecording_prepreDay['run'].iloc[list(np.where(1==1*(runRecording_prepreDay['type']=='recognition'))[0])])[-(8-len(actualRuns)-len(actualRuns_preDay)):] # might be [5,6,7,8]
else:
    actualRuns_prepreDay = []

objects = ['bed', 'bench', 'chair', 'table']

for ii,run in enumerate(actualRuns): # load behavior and brain data for current session
    t = np.load(f"{cfg.recognition_dir}brain_run{run}.npy")
    mask = np.load(f"{cfg.chosenMask}")
    t = t[:,mask==1]
    t = normalize(t)
    brain_data=t if ii==0 else np.concatenate((brain_data,t), axis=0)

    t = pd.read_csv(f"{cfg.recognition_dir}behav_run{run}.csv")
    behav_data=t if ii==0 else pd.concat([behav_data,t])

for ii,run in enumerate(actualRuns_preDay): # load behavior and brain data for previous session
    t = np.load(f"{cfg.subjects_dir}{cfg.subjectName}/ses{cfg.session-1}/recognition/brain_run{run}.npy")
    mask = np.load(f"{cfg.chosenMask}")
    t = t[:,mask==1]
    t = normalize(t)
    brain_data = np.concatenate((brain_data,t), axis=0)

    t = pd.read_csv(f"{cfg.subjects_dir}{cfg.subjectName}/ses{cfg.session-1}/recognition/behav_run{run}.csv")
    behav_data = pd.concat([behav_data,t])

for ii,run in enumerate(actualRuns_prepreDay): # load behavior and brain data for previous session
    t = np.load(f"{cfg.subjects_dir}{cfg.subjectName}/ses{cfg.session-2}/recognition/brain_run{run}.npy")
    mask = np.load(f"{cfg.chosenMask}")
    t = t[:,mask==1]
    t = normalize(t)
    brain_data = np.concatenate((brain_data,t), axis=0)

    t = pd.read_csv(f"{cfg.subjects_dir}{cfg.subjectName}/ses{cfg.session-2}/recognition/behav_run{run}.csv")
    behav_data = pd.concat([behav_data,t])

# FEAT=brain_data.reshape(brain_data.shape[0],-1)

FEAT=brain_data
print(f"FEAT.shape={FEAT.shape}")
assert len(FEAT.shape)==2
# FEAT_mean=np.mean(FEAT,axis=1)
# FEAT=(FEAT.T-FEAT_mean).T
# FEAT_mean=np.mean(FEAT,axis=0)
# FEAT=FEAT-FEAT_mean
# FEAT = normalize(FEAT)

META=behav_data

# convert item colume to label colume
imcodeDict={
'A': 'bed',
'B': 'chair',
'C': 'table',
'D': 'bench'}
label=[]
for curr_trial in range(META.shape[0]):
    label.append(imcodeDict[META['Item'].iloc[curr_trial]])
META['label']=label # merge the label column with the data dataframe

# Which run to use as test data (leave as None to not have test data)
testRun = 1 # when testing: testRun = 2 ; META['run_num'].iloc[:5]=2

# Decide on the proportion of crescent data to use for classification
allpairs = itertools.combinations(objects,2)
accs={}
# Iterate over all the possible target pairs of objects
for pair in allpairs:
    # Find the control (remaining) objects for this pair
    altpair = other(pair)
    
    # pull sorted indices for each of the critical objects, in order of importance (low to high)
    # inds = get_inds(FEAT, META, pair, testRun=testRun)
    
    # Find the number of voxels that will be left given your inclusion parameter above
    # nvox = red_vox(FEAT.shape[1], include)
    
    for obj in pair:
        # foil = [i for i in pair if i != obj][0]
        for altobj in altpair:
            
            # establish a naming convention where it is $TARGET_$CLASSIFICATION
            # Target is the NF pair (e.g. bed/bench)
            # Classificationis is btw one of the targets, and a control (e.g. bed/chair, or bed/table, NOT bed/bench)
            naming = '{}{}_{}{}'.format(pair[0], pair[1], obj, altobj)
            
            # Pull the relevant inds from your previously established dictionary 
            # obj_inds = inds[obj]
            
            # If you're using testdata, this function will split it up. Otherwise it leaves out run as a parameter
            # if testRun:
            #     trainIX = META.index[(META['label'].isin([obj, altobj])) & (META['run_num'] != int(testRun))]
            #     testIX = META.index[(META['label'].isin([obj, altobj])) & (META['run_num'] == int(testRun))]
            # else:
            #     trainIX = META.index[(META['label'].isin([obj, altobj]))]
            #     testIX = META.index[(META['label'].isin([obj, altobj]))]
            # # pull training and test data
            # trainX = FEAT[trainIX]
            # testX = FEAT[testIX]
            # trainY = META.iloc[trainIX].label
            # testY = META.iloc[testIX].label
            
            # print(f"obj={obj},altobj={altobj}")
            # print(f"unique(trainY)={np.unique(trainY)}")
            # print(f"unique(testY)={np.unique(testY)}")
            # assert len(np.unique(trainY))==2
            from sklearn.model_selection import GridSearchCV, PredefinedSplit
            for testRun in range(1,7):
                if testRun:
                    trainIX = ((META['label']==obj) + (META['label']==altobj)) * (META['run_num']!=int(testRun))
                    testIX = ((META['label']==obj) + (META['label']==altobj)) * (META['run_num']==int(testRun))
                else:
                    trainIX = ((META['label']==obj) | (META['label']==altobj))
                    testIX = ((META['label']==obj) | (META['label']==altobj))
                # pull training and test data
                trainX = FEAT[trainIX]
                testX = FEAT[testIX]
                trainY = META.iloc[np.asarray(trainIX)].label
                testY = META.iloc[np.asarray(testIX)].label

                # print(f"obj={obj},altobj={altobj}")
                # print(f"unique(trainY)={np.unique(trainY)}")
                # print(f"unique(testY)={np.unique(testY)}")
                assert len(np.unique(trainY))==2
                error
                # # If you're selecting high-importance features, this bit handles that
                # if include < 1:
                #     trainX = trainX[:, obj_inds[-nvox:]]
                #     testX = testX[:, obj_inds[-nvox:]]
                

FEAT.shape=(384, 546)


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/scipy/stats/stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in P

NameError: name 'error' is not defined

In [187]:
clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
                            multi_class='multinomial').fit(trainX, trainY)

model_folder = cfg.trainingModel_dir
# Save it for later use
# joblib.dump(clf, model_folder +'/{}.joblib'.format(naming))

# Monitor progress by printing accuracy (only useful if you're running a test set)
acc = clf.score(testX, testY)
print("testing accuracy = ",end='')
print(naming, acc)
accs[naming]=acc




# parameters = {'penalty':('l2', 'elasticnet'), 'C':[1, 0.5,0.2,0.1,0.01]}
# logi = LogisticRegression()
# clf = GridSearchCV(logi, parameters)

# clf.fit(trainX, trainY)
# # # Train your classifier
# # clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
# #                             multi_class='multinomial').fit(trainX, trainY)

# # model_folder = cfg.trainingModel_dir
# # Save it for later use
# # joblib.dump(clf, model_folder +'/{}.joblib'.format(naming))

# # Monitor progress by printing accuracy (only useful if you're running a test set)
# acc = clf.score(testX, testY)
# print("testing accuracy = ",end='')
# print(naming, acc)


testing accuracy = bedbench_bedchair 0.8333333333333334


In [188]:
testRun=1
if testRun:
    trainIX = ((META['label']==obj) + (META['label']==altobj)) * (META['run_num']!=int(testRun))
    testIX = ((META['label']==obj) + (META['label']==altobj)) * (META['run_num']==int(testRun))
else:
    trainIX = ((META['label']==obj) | (META['label']==altobj))
    testIX = ((META['label']==obj) | (META['label']==altobj))
# pull training and test data
trainX = FEAT[trainIX]
testX = FEAT[testIX]
trainY = META.iloc[np.asarray(trainIX)].label
testY = META.iloc[np.asarray(testIX)].label

# print(f"obj={obj},altobj={altobj}")
# print(f"unique(trainY)={np.unique(trainY)}")
# print(f"unique(testY)={np.unique(testY)}")
assert len(np.unique(trainY))==2

# # If you're selecting high-importance features, this bit handles that
# if include < 1:
#     trainX = trainX[:, obj_inds[-nvox:]]
#     testX = testX[:, obj_inds[-nvox:]]

parameters = {'penalty':('l2', 'elasticnet'), 'C':[1, 0.5,0.2,0.1,0.01]}
logi = LogisticRegression()
clf = GridSearchCV(logi, parameters,cv=6)

clf.fit(trainX, trainY)
# # Train your classifier
# clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
#                             multi_class='multinomial').fit(trainX, trainY)

# model_folder = cfg.trainingModel_dir
# Save it for later use
# joblib.dump(clf, model_folder +'/{}.joblib'.format(naming))

# Monitor progress by printing accuracy (only useful if you're running a test set)
acc = clf.score(testX, testY)
print("testing accuracy = ",end='')
print(naming, acc)
accs[naming]=acc


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/use

testing accuracy = bedbench_bedchair 0.8333333333333334


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got el

In [81]:
def behaviorDataLoading(cfg,curr_run):
    '''
    extract the labels which is selected by the subject and coresponding TR and time
    check if the subject's response is correct. When Item is A,bed, response should be 1, or it is wrong
    '''
    behav_data = pd.read_csv(f"{cfg.recognition_dir}{cfg.subjectName}_{curr_run}.csv")

    # the item(imcode) colume of the data represent each image in the following correspondence
    imcodeDict={
    'A': 'bed',
    'B': 'chair',
    'C': 'table',
    'D': 'bench'}

    # When the imcode code is "A", the correct response should be '1', "B" should be '2'
    correctResponseDict={
    'A': 1,
    'B': 2,
    'C': 1,
    'D': 2}

    # extract the labels which is selected by the subject and coresponding TR and time
    behav_data = behav_data[['TR', 'image_on', 'Resp',  'Item']] # the TR, the real time it was presented, 

    # this for loop is to deal with the situation where Resp is late for 1 TR, or two buttons are pressed. 
    # when Resp is late for 1 TR, set the current Resp as the later Response.
    # when two buttons are pressed, set the current Resp as the later Response because the later one should be the real choice
    for curr_trial in range(behav_data.shape[0]):
        if behav_data['Item'].iloc[curr_trial]  in ["A","B","C","D"]:
            if behav_data['Resp'].iloc[curr_trial+1] in [1.0,2.0]:
                behav_data['Resp'].iloc[curr_trial]=behav_data['Resp'].iloc[curr_trial+1]

    behav_data=behav_data.dropna(subset=['Item'])

    # check if the subject's response is correct. When Item is A,bed, response should be 1, or it is wrong
    isCorrect=[]
    for curr_trial in range(behav_data.shape[0]):
        isCorrect.append(correctResponseDict[behav_data['Item'].iloc[curr_trial]]==behav_data['Resp'].iloc[curr_trial])
    print(f"accuracy for run {curr_run} = {np.mean(isCorrect)}")

    behav_data['isCorrect']=isCorrect # merge the isCorrect clumne with the data dataframe
    behav_data['subj']=[cfg.subjectName for i in range(len(behav_data))]
    behav_data['run_num']=[int(curr_run) for i in range(len(behav_data))]
    behav_data=behav_data[behav_data['isCorrect']] # discard the trials where the subject made wrong selection
    return behav_data

In [87]:
curr_run_behav=0
behav_data = behaviorDataLoading(cfg,curr_run_behav+1)
behav_data.shape

accuracy for run 1 = 1.0


(48, 7)

In [84]:
runRecording = pd.read_csv(f"{cfg.recognition_dir}../runRecording.csv")
actualRuns = list(runRecording['run'].iloc[list(np.where(1==1*(runRecording['type']=='recognition'))[0])])[:2]


[2, 3]

In [ ]:
curr_run=2

brain_data = nib.load(f"{cfg.recognition_dir}run{curr_run}.nii.gz").get_data() ; brain_data=np.transpose(brain_data,(3,0,1,2))
Brain_TR=np.arange(brain_data.shape[0])
Brain_TR = Brain_TR+2

# select volumes of brain_data by counting which TR is left in behav_data
Brain_TR=Brain_TR[list(behav_data['TR'])] # original TR begin with 0
if Brain_TR[-1]>=brain_data.shape[0]: # when the brain data is not as long as the behavior data, delete the last row
    Brain_TR = Brain_TR[:-1]
    behav_data = behav_data.drop([behav_data.iloc[-1].TR])
brain_data=brain_data[Brain_TR]

In [102]:
choose = np.load(f"{cfg.subjects_dir}/{cfg.subjectName}/ses{cfg.session}/recognition/choose.npy")
order = f'{cfg.recognition_expScripts_dir}/orders/recognitionOrders_{choose[1 - 1]}.csv'
trial_list = pd.read_csv(order)
trial_list



,Unnamed: 0,time,imnum,dur,weight,imcode,path,corrAns,axis,button_left,button_right,viewPoint,imgPath
0,0,6,3.0,1.0,1,D,NaN,100,tableBench,Table,Bench,33,./carchair_exp/tableBench_100_33.png
1,1,10,1.0,1.0,1,B,NaN,100,bedChair,Bed,Chair,30,./carchair_exp/bedChair_100_30.png
2,2,16,0.0,1.0,1,A,NaN,1,bedChair,Bed,Chair,24,./carchair_exp/bedChair_1_24.png
3,3,24,2.0,1.0,1,C,NaN,1,tableBench,Table,Bench,3,./carchair_exp/tableBench_1_3.png
4,4,30,3.0,1.0,1,D,NaN,100,tableBench,Table,Bench,12,./carchair_exp/tableBench_100_12.png
5,5,36,2.0,1.0,1,C,NaN,1,tableBench,Table,Bench,6,./carchair_exp/tableBench_1_6.png
6,6,42,1.0,1.0,1,B,NaN,100,bedChair,Bed,Chair,15,./carchair_exp/bedChair_100_15.png
7,7,48,3.0,1.0,1,D,NaN,100,tableBench,Table,Bench,18,./carchair_exp/tableBench_100_18.png
8,8,52,2.0,1.0,1,C,NaN,1,tableBench,Table,Bench,12,./carchair_exp/tableBench_1_12.png
9,9,56,0.0,1.0,1,A,NaN,1,bedChair,Bed,Chair,9,./carchair_exp/bedChair_1_9.png


In [104]:
ims = list(trial_list['imcode'])
imcodeDict={
'A': 'bed',
'B': 'chair',
'C': 'table',
'D': 'bench'}

In [105]:
imcodeDict["A"]

'bed'

In [ ]:
correctResponseDict={
'A': 1,
'B': 2,
'C': 1,
'D': 2}

In [112]:
subjects_correctly_aligned=['1206161','0119173','1206162','1130161','1206163','0120171','0111171','1202161','0125172','0110172','0123173','0120173','0110171','0119172','0124171','0123171','1203161','0118172','0118171','0112171','1207162','0117171','0119174','0112173','0112172']
subjects=subjects_correctly_aligned


[i for i,x in enumerate(subjects) if x == "0119173"][0]

1

In [129]:

def getMask(topN, cfg):
    for pn, parc in enumerate(topN):
        _mask = nib.load(cfg.recognition_dir+"mask/schaefer_{}".format(parc))
        # schaefer_56.nii.gz
        aff = _mask.affine
        _mask = _mask.get_data()
        _mask = _mask.astype(int)
        # say some things about the mask.
        mask = _mask if pn == 0 else mask + _mask
        mask[mask>0] = 1
    return mask
roiloc="schaefer2018"
dataSource="realtime"
subjects=[cfg.subjectName]
N=25
GreedyBestAcc=np.zeros((len(subjects),N+1))
GreedyBestAcc[GreedyBestAcc==0]=None
for ii,subject in enumerate(subjects):
#     try:
#         GreedyBestAcc[ii,40]=np.load("./{}/{}/output/top{}.npy".format(roiloc, subject, N))
#     except:
#         pass
    for len_topN_1 in range(N-1,0,-1):
        try:
            # print(f"./tmp_folder/{subject}_{N}_{roiloc}_{dataSource}_{len_topN_1}")
            di = load_obj(f"./tmp_folder/{subject}_{N}_{roiloc}_{dataSource}_{len_topN_1}")
            GreedyBestAcc[ii,len_topN_1-1] = di['bestAcc']
        except:
            pass
GreedyBestAcc=GreedyBestAcc.T

# plt.imshow(GreedyBestAcc)
# _=plt.figure()
# for i in range(GreedyBestAcc.shape[0]):
#     plt.scatter([i]*GreedyBestAcc.shape[1],GreedyBestAcc[i],c='g',s=2)
# plt.plot(np.arange(GreedyBestAcc.shape[0]),np.nanmean(GreedyBestAcc,axis=1))

performance_mean = np.nanmean(GreedyBestAcc,axis=1)
bestID=np.where(performance_mean==max(performance_mean))[0][0]
di = load_obj(f"./tmp_folder/{subject}_{N}_{roiloc}_{dataSource}_{bestID+1}")
mask = getMask(di['bestROIs'],cfg)
di

/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/ipykernel_launcher.py:38: RuntimeWarning: Mean of empty slice
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  


{'subject': 'sub001',
 'startFromN': 25,
 'currNumberOfROI': 5,
 'bestAcc': array(0.47135417),
 'bestROIs': ('19.nii.gz',
  '232.nii.gz',
  '102.nii.gz',
  '86.nii.gz',
  '167.nii.gz')}

In [135]:

from scipy.stats import zscore
def normalize(X):
    _X=X.copy()
    _X = zscore(_X, axis=0)
    _X[np.isnan(_X)]=0
    return _X

def mkdir(folder):
    if not os.path.isdir(folder):
        os.mkdir(folder)

'''
calculate the evidence floor and ceil for each subject and display different forms of evidences.
'''
def morphingTarget(subject):
    '''
    purpose:
        get the morphing target function
    steps:
        load train clf
        load brain data and behavior data
        get the morphing target function
            evidence_floor is C evidence for CD classifier(can also be D evidence for CD classifier)
            evidence_ceil  is A evidence in AC and AD classifier
    '''

    import os
    import numpy as np
    import pandas as pd
    import joblib
    import nibabel as nib


    phasedict = dict(zip([1,2,3,4,5,6],["12", "12", "34", "34", "56", "56"]))
    imcodeDict={"A": "bed", "B": "Chair", "C": "table", "D": "bench"}
    if 'milgram' in os.getcwd():
        main_dir='/gpfs/milgram/project/turk-browne/projects/rtTest/'
    else:
        main_dir='/Users/kailong/Desktop/rtTest'

    working_dir=main_dir
    os.chdir(working_dir)

    funcdata = "/gpfs/milgram/project/turk-browne/jukebox/ntb/projects/sketchloop02/subjects/{sub}_neurosketch/data/nifti/realtime_preprocessed/{sub}_neurosketch_recognition_run_{run}.nii.gz"
    metadata = "/gpfs/milgram/project/turk-browne/jukebox/ntb/projects/sketchloop02/data/features/recog/metadata_{sub}_V1_{phase}.csv"

    metas = []
    for run in range(1, 7):
        print(run, end='--')
        # retrieve from the dictionary which phase it is, assign the session
        phase = phasedict[run]
        ses = 1
        
        # Build the path for the preprocessed functional data
        this4d = funcdata.format(ses=ses, run=run, phase=phase, sub=subject)
        
        # Read in the metadata, and reduce it to only the TR values from this run, add to a list
        thismeta = pd.read_csv(metadata.format(ses=ses, run=run, phase=phase, sub=subject))
        if dataSource == "neurosketch":
            _run = 1 if run % 2 == 0 else 2
        else:
            _run = run
        thismeta = thismeta[thismeta['run_num'] == int(_run)]
        
        if dataSource == "realtime":
            TR_num = list(thismeta.TR.astype(int))
            labels = list(thismeta.Item)
            labels = [imcodeDict[label] for label in labels]
        else:
            TR_num = list(thismeta.TR_num.astype(int))
            labels = list(thismeta.label)
        
        print("LENGTH OF TR: {}".format(len(TR_num)))
        # Load the functional data
        runIm = nib.load(this4d)
        affine_mat = runIm.affine
        runImDat = runIm.get_fdata()
        
        # Use the TR numbers to select the correct features
        features = [runImDat[:,:,:,n+3] for n in TR_num]
        features = np.array(features)
        chosenMask = np.load(f"/gpfs/milgram/project/turk-browne/projects/rtTest/schaefer2018/{subject}/chosenMask.npy")
        features = features[:, chosenMask==1]
        print("shape of features", features.shape, "shape of mask", mask.shape)
        # featmean = features.mean(1).mean(1).mean(1)[..., None,None,None] #features.mean(1)[..., None]
        # features = features - featmean
        # features = features - features.mean(0)
        features = normalize(features)
        # features = np.expand_dims(features, 0)
        
        # Append both so we can use it later
        # metas.append(labels)
        # metas['label']

        t=pd.DataFrame()
        t['label']=labels
        t["run_num"]=run
        behav_data=t if run==1 else pd.concat([behav_data,t])
        
        runs = features if run == 1 else np.concatenate((runs, features))

    dimsize = runIm.header.get_zooms()
    
    brain_data = runs
    print(brain_data.shape)
    print(behav_data.shape)
    FEAT=brain_data
    print(f"FEAT.shape={FEAT.shape}")
    META=behav_data

    # print('mask dimensions: {}'. format(mask.shape))
    # print('number of voxels in mask: {}'.format(np.sum(mask)))

    # runRecording = pd.read_csv(f"{cfg.recognition_dir}../runRecording.csv")
    # actualRuns = list(runRecording['run'].iloc[list(np.where(1==1*(runRecording['type']=='recognition'))[0])]) # can be [1,2,3,4,5,6,7,8] or [1,2,4,5]

    # objects = ['bed', 'bench', 'chair', 'table']

    # for ii,run in enumerate(actualRuns[:2]): # load behavior and brain data for current session
    #     t = np.load(f"{cfg.recognition_dir}brain_run{run}.npy")
    #     # mask = nib.load(f"{cfg.chosenMask}").get_data()
    #     mask = np.load(cfg.chosenMask)
    #     t = t[:,mask==1]
    #     t = normalize(t)
    #     brain_data=t if ii==0 else np.concatenate((brain_data,t), axis=0)

    #     t = pd.read_csv(f"{cfg.recognition_dir}behav_run{run}.csv")
    #     behav_data=t if ii==0 else pd.concat([behav_data,t])

    # FEAT=brain_data.reshape(brain_data.shape[0],-1)
    # # FEAT_mean=np.mean(FEAT,axis=1)
    # # FEAT=(FEAT.T-FEAT_mean).T
    # # FEAT_mean=np.mean(FEAT,axis=0)
    # # FEAT=FEAT-FEAT_mean

    # META=behav_data

    # convert item colume to label colume
    imcodeDict={
    'A': 'bed',
    'B': 'chair',
    'C': 'table',
    'D': 'bench'}
    # label=[]
    # for curr_trial in range(META.shape[0]):
    #     label.append(imcodeDict[META['Item'].iloc[curr_trial]])
    # META['label']=label # merge the label column with the data dataframe


    # def classifierEvidence(clf,X,Y): # X shape is [trials,voxelNumber], Y is ['bed', 'bed'] for example # return a 1-d array of probability
    #     # This function get the data X and evidence object I want to know Y, and output the trained model evidence.
    #     targetID=[np.where((clf.classes_==i)==True)[0][0] for i in Y]
    #     # Evidence=(np.sum(X*clf.coef_,axis=1)+clf.intercept_) if targetID[0]==1 else (1-(np.sum(X*clf.coef_,axis=1)+clf.intercept_))
    #     Evidence=(X@clf.coef_.T+clf.intercept_) if targetID[0]==1 else (-(X@clf.coef_.T+clf.intercept_))
    #     Evidence = 1/(1+np.exp(-Evidence))
    #     return np.asarray(Evidence)

    # def classifierEvidence(clf,X,Y):
    #     ID=np.where((clf.classes_==Y[0])*1==1)[0][0]
    #     p = clf.predict_proba(X)[:,ID]
    #     BX=np.log(p/(1-p))
    #     return BX

    def classifierEvidence(clf,X,Y):
        ID=np.where((clf.classes_==Y[0])*1==1)[0][0]
        Evidence=(X@clf.coef_.T+clf.intercept_) if ID==1 else (-(X@clf.coef_.T+clf.intercept_))
        # Evidence=(X@clf.coef_.T+clf.intercept_) if ID==0 else (-(X@clf.coef_.T+clf.intercept_))
        return np.asarray(Evidence)

    A_ID = (META['label']=='bed')
    X = FEAT[A_ID]

    # evidence_floor is C evidence for AC_CD BC_CD CD_CD classifier(can also be D evidence for CD classifier)
    # Y = ['table'] * X.shape[0]
    # CD_clf=joblib.load(cfg.usingModel_dir +'bedbench_benchtable.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
    # CD_C_evidence = classifierEvidence(CD_clf,X,Y)
    # evidence_floor = np.mean(CD_C_evidence)
    # print(f"evidence_floor={evidence_floor}")

    model_folder = f"{working_dir}{roiloc}/{subject}/clf/"

    # #try out other forms of floor: C evidence in AC and D evidence for AD
    # Y = ['bench'] * X.shape[0]
    # AD_clf=joblib.load(model_folder +'bedchair_bedbench.joblib') # These 4 clf are the same:   bedchair_bedbench.joblib bedtable_bedbench.joblib benchchair_benchbed.joblib benchtable_benchbed.joblib
    # AD_D_evidence = classifierEvidence(AD_clf,X,Y)
    # evidence_floor = np.mean(AD_D_evidence)
    # print(f"evidence_floor2={np.mean(evidence_floor)}")



    # # floor
    # Y = ['bench'] * X.shape[0]
    # CD_clf=joblib.load(model_folder +'bedbench_benchtable.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
    # CD_D_evidence = classifierEvidence(CD_clf,X,Y)
    # evidence_floor = np.mean(CD_D_evidence)
    # print(f"evidence_floor={evidence_floor}")

    # Y = ['table'] * X.shape[0]
    # CD_clf=joblib.load(model_folder +'bedbench_benchtable.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
    # CD_C_evidence = classifierEvidence(CD_clf,X,Y)
    # evidence_floor = np.mean(CD_C_evidence)
    # print(f"evidence_floor={evidence_floor}")


    # # evidence_ceil  is A evidence in AC and AD classifier
    # Y = ['bed'] * X.shape[0]
    # AC_clf=joblib.load(model_folder +'benchtable_tablebed.joblib') # These 4 clf are the same:   bedbench_bedtable.joblib bedchair_bedtable.joblib benchtable_tablebed.joblib chairtable_tablebed.joblib
    # AC_A_evidence = classifierEvidence(AC_clf,X,Y)
    # evidence_ceil1 = AC_A_evidence
    # print(f"evidence_ceil1={np.mean(evidence_ceil1)}")

    # Y = ['bed'] * X.shape[0]
    # AD_clf=joblib.load(model_folder +'bedchair_bedbench.joblib') # These 4 clf are the same:   bedchair_bedbench.joblib bedtable_bedbench.joblib benchchair_benchbed.joblib benchtable_benchbed.joblib
    # AD_A_evidence = classifierEvidence(AD_clf,X,Y)
    # evidence_ceil2 = AD_A_evidence
    # print(f"evidence_ceil2={np.mean(evidence_ceil2)}")

    # # evidence_ceil = np.mean(evidence_ceil1)
    # # evidence_ceil = np.mean(evidence_ceil2)
    # evidence_ceil = np.mean((evidence_ceil1+evidence_ceil2)/2)
    # print(f"evidence_ceil={evidence_ceil}")
    store="\n"
    print("floor")
    # D evidence for AD_clf when A is presented.
    Y = ['bench'] * X.shape[0]
    AD_clf=joblib.load(model_folder +'bedchair_bedbench.joblib') # These 4 clf are the same:   bedchair_bedbench.joblib bedtable_bedbench.joblib benchchair_benchbed.joblib benchtable_benchbed.joblib
    AD_D_evidence = classifierEvidence(AD_clf,X,Y)
    evidence_floor = np.mean(AD_D_evidence)
    print(f"D evidence for AD_clf when A is presented={evidence_floor}")
    store=store+f"D evidence for AD_clf when A is presented={evidence_floor}"

    # C evidence for AC_clf when A is presented.
    Y = ['table'] * X.shape[0]
    AC_clf=joblib.load(model_folder +'benchtable_tablebed.joblib') # These 4 clf are the same:   bedbench_bedtable.joblib bedchair_bedtable.joblib benchtable_tablebed.joblib chairtable_tablebed.joblib
    AC_C_evidence = classifierEvidence(AC_clf,X,Y)
    evidence_floor = np.mean(AC_C_evidence)
    print(f"C evidence for AC_clf when A is presented={evidence_floor}")
    store=store+"\n"+f"C evidence for AC_clf when A is presented={evidence_floor}"

    # D evidence for CD_clf when A is presented.
    Y = ['bench'] * X.shape[0]
    CD_clf=joblib.load(model_folder +'bedbench_benchtable.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
    CD_D_evidence = classifierEvidence(CD_clf,X,Y)
    evidence_floor = np.mean(CD_D_evidence)
    print(f"D evidence for CD_clf when A is presented={evidence_floor}")
    store=store+"\n"+f"D evidence for CD_clf when A is presented={evidence_floor}"

    # C evidence for CD_clf when A is presented.
    Y = ['table'] * X.shape[0]
    CD_clf=joblib.load(model_folder +'bedbench_benchtable.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
    CD_C_evidence = classifierEvidence(CD_clf,X,Y)
    evidence_floor = np.mean(CD_C_evidence)
    print(f"C evidence for CD_clf when A is presented={evidence_floor}")
    store=store+"\n"+f"C evidence for CD_clf when A is presented={evidence_floor}"




    print("ceil")
    store=store+"\n"+"ceil"
    # evidence_ceil  is A evidence in AC and AD classifier
    Y = ['bed'] * X.shape[0]
    AC_clf=joblib.load(model_folder +'benchtable_tablebed.joblib') # These 4 clf are the same:   bedbench_bedtable.joblib bedchair_bedtable.joblib benchtable_tablebed.joblib chairtable_tablebed.joblib
    AC_A_evidence = classifierEvidence(AC_clf,X,Y)
    evidence_ceil1 = AC_A_evidence
    print(f"A evidence in AC_clf when A is presented={np.mean(evidence_ceil1)}")
    store=store+"\n"+f"A evidence in AC_clf when A is presented={np.mean(evidence_ceil1)}"

    Y = ['bed'] * X.shape[0]
    AD_clf=joblib.load(model_folder +'bedchair_bedbench.joblib') # These 4 clf are the same:   bedchair_bedbench.joblib bedtable_bedbench.joblib benchchair_benchbed.joblib benchtable_benchbed.joblib
    AD_A_evidence = classifierEvidence(AD_clf,X,Y)
    evidence_ceil2 = AD_A_evidence
    print(f"A evidence in AD_clf when A is presented={np.mean(evidence_ceil2)}")
    store=store+"\n"+f"A evidence in AD_clf when A is presented={np.mean(evidence_ceil2)}"

    # evidence_ceil = np.mean(evidence_ceil1)
    # evidence_ceil = np.mean(evidence_ceil2)
    evidence_ceil = np.mean((evidence_ceil1+evidence_ceil2)/2)
    print(f"evidence_ceil={evidence_ceil}")
    store=store+"\n"+f"evidence_ceil={evidence_ceil}"

    return evidence_floor, evidence_ceil,store
subjects_correctly_aligned=['1206161','0119173','1206162','1130161','1206163','0120171','0111171','1202161','0125172','0110172','0123173','0120173','0110171','0119172','0124171','0123171','1203161','0118172','0118171','0112171','1207162','0117171','0119174','0112173','0112172']
subjects=subjects_correctly_aligned
for subject in subjects:
    floor, ceil,store = morphingTarget(subject)
    mu = (ceil+floor)/2
    sig = (ceil-floor)/2.3548
    print(f"floor={floor}, ceil={ceil}")
    print(f"mu={mu}, sig={sig}")

    store=store+"\n"+f"floor={floor}, ceil={ceil}"
    store=store+"\n"+f"mu={mu}, sig={sig}"

1--LENGTH OF TR: 80
shape of features (80, 4954) shape of mask (64, 64, 36)
2--LENGTH OF TR: 80
shape of features (80, 4954) shape of mask (64, 64, 36)
3--LENGTH OF TR: 80
shape of features (80, 4954) shape of mask (64, 64, 36)
4--LENGTH OF TR: 80
shape of features (80, 4954) shape of mask (64, 64, 36)
5--LENGTH OF TR: 80
shape of features (80, 4954) shape of mask (64, 64, 36)
6--LENGTH OF TR: 80
shape of features (80, 4954) shape of mask (64, 64, 36)
(480, 4954)
(480, 2)
FEAT.shape=(480, 4954)
floor
D evidence for AD_clf when A is presented=-2.8479903196091962
C evidence for AC_clf when A is presented=-2.8566487949601025
D evidence for CD_clf when A is presented=-0.0007392565968442732
C evidence for CD_clf when A is presented=0.0007392565968442732
ceil
A evidence in AC_clf when A is presented=2.8566487949601025
A evidence in AD_clf when A is presented=2.8479903196091962
evidence_ceil=2.8523195572846496
floor=0.0007392565968442732, ceil=2.8523195572846496
mu=1.426529406940747, sig=1.21

/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/scipy/stats/stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


shape of features (80, 3294) shape of mask (64, 64, 36)
4--LENGTH OF TR: 80


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/scipy/stats/stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


shape of features (80, 3294) shape of mask (64, 64, 36)
5--LENGTH OF TR: 80
shape of features (80, 3294) shape of mask (64, 64, 36)
6--LENGTH OF TR: 80


...

shape of features (80, 4915) shape of mask (64, 64, 36)
5--LENGTH OF TR: 80
shape of features (80, 4915) shape of mask (64, 64, 36)
6--LENGTH OF TR: 80


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/scipy/stats/stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


shape of features (80, 4915) shape of mask (64, 64, 36)
(480, 4915)
(480, 2)
FEAT.shape=(480, 4915)
floor
D evidence for AD_clf when A is presented=-2.9551856637416813
C evidence for AC_clf when A is presented=-2.99875213582018
D evidence for CD_clf when A is presented=0.117411710458363
C evidence for CD_clf when A is presented=-0.117411710458363
ceil
A evidence in AC_clf when A is presented=2.99875213582018
A evidence in AD_clf when A is presented=2.9551856637416813
evidence_ceil=2.9769688997809305
floor=-0.117411710458363, ceil=2.9769688997809305
mu=1.4297785946612838, sig=1.3140736411751712
1--LENGTH OF TR: 80


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/scipy/stats/stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


shape of features (80, 2254) shape of mask (64, 64, 36)
2--LENGTH OF TR: 80
shape of features (80, 2254) shape of mask (64, 64, 36)
3--LENGTH OF TR: 80
shape of features (80, 2254) shape of mask (64, 64, 36)
4--LENGTH OF TR: 80
shape of features (80, 2254) shape of mask (64, 64, 36)
5--LENGTH OF TR: 80
shape of features (80, 2254) shape of mask (64, 64, 36)
6--LENGTH OF TR: 80
shape of features (80, 2254) shape of mask (64, 64, 36)
(480, 2254)
(480, 2)
FEAT.shape=(480, 2254)
floor
D evidence for AD_clf when A is presented=-2.6276704099217683
C evidence for AC_clf when A is presented=-2.648179686283126
D evidence for CD_clf when A is presented=0.07434422271153115
C evidence for CD_clf when A is presented=-0.07434422271153115
ceil
A evidence in AC_clf when A is presented=2.648179686283126
A evidence in AD_clf when A is presented=2.6276704099217683
evidence_ceil=2.6379250481024474
floor=-0.07434422271153115, ceil=2.6379250481024474
mu=1.2817904126954582, sig=1.1518045145294626
1--LENGTH O

In [137]:
# def minimalClass(cfg):
'''
purpose: 
    train offline models

steps:
    load preprocessed and aligned behavior and brain data 
    select data with the wanted pattern like AB AC AD BC BD CD 
    train correspondng classifier and save the classifier performance and the classifiers themselves.
'''

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import joblib
import nibabel as nib
import itertools
from sklearn.linear_model import LogisticRegression

def gaussian(x, mu, sig):
    # mu and sig is determined before each neurofeedback session using 2 recognition runs.
    return round(1+18*(1 - np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))))) # map from (0,1) -> [1,19]

def jitter(size,const=0):
    jit = np.random.normal(0+const, 0.05, size)
    X = np.zeros((size))
    X = X + jit
    return X

def other(target):
    other_objs = [i for i in ['bed', 'bench', 'chair', 'table'] if i not in target]
    return other_objs

def red_vox(n_vox, prop=0.1):
    return int(np.ceil(n_vox * prop))

def get_inds(X, Y, pair, testRun=None):

    inds = {}

    # return relative indices
    if testRun:
        trainIX = Y.index[(Y['label'].isin(pair)) & (Y['run_num'] != int(testRun))]
    else:
        trainIX = Y.index[(Y['label'].isin(pair))]

    # pull training and test data
    trainX = X[trainIX]
    trainY = Y.iloc[trainIX].label

    # Main classifier on 5 runs, testing on 6th
    clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
                             multi_class='multinomial').fit(trainX, trainY)
    B = clf.coef_[0]  # pull betas

    # retrieve only the first object, then only the second object
    if testRun:
        obj1IX = Y.index[(Y['label'] == pair[0]) & (Y['run_num'] != int(testRun))]
        obj2IX = Y.index[(Y['label'] == pair[1]) & (Y['run_num'] != int(testRun))]
    else:
        obj1IX = Y.index[(Y['label'] == pair[0])]
        obj2IX = Y.index[(Y['label'] == pair[1])]

    # Get the average of the first object, then the second object
    obj1X = np.mean(X[obj1IX], 0)
    obj2X = np.mean(X[obj2IX], 0)

    # Build the importance map
    mult1X = obj1X * B
    mult2X = obj2X * B

    # Sort these so that they are from least to most important for a given category.
    sortmult1X = mult1X.argsort()[::-1]
    sortmult2X = mult2X.argsort()

    # add to a dictionary for later use
    inds[clf.classes_[0]] = sortmult1X
    inds[clf.classes_[1]] = sortmult2X

    return inds

if 'milgram' in os.getcwd():
    main_dir='/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/'
else:
    main_dir='/Volumes/GoogleDrive/My Drive/Turk_Browne_Lab/rtcloud_kp/'

working_dir=main_dir
os.chdir(working_dir)

'''
if you read runRecording for current session and found that there are only 4 runs in the current session, 
you read the runRecording for previous session and fetch the last 4 recognition runs from previous session
'''
runRecording = pd.read_csv(f"{cfg.recognition_dir}../runRecording.csv")
actualRuns = list(runRecording['run'].iloc[list(np.where(1==1*(runRecording['type']=='recognition'))[0])]) # can be [1,2,3,4,5,6,7,8] or [1,2,4,5]
if len(actualRuns) < 8:
    runRecording_preDay = pd.read_csv(f"{cfg.subjects_dir}{cfg.subjectName}/ses{cfg.session-1}/recognition/../runRecording.csv")
    actualRuns_preDay = list(runRecording_preDay['run'].iloc[list(np.where(1==1*(runRecording_preDay['type']=='recognition'))[0])])[-(8-len(actualRuns)):] # might be [5,6,7,8]
else: 
    actualRuns_preDay = []

# assert len(actualRuns_preDay)+len(actualRuns)==8 
if len(actualRuns_preDay)+len(actualRuns)<8:
    runRecording_prepreDay = pd.read_csv(f"{cfg.subjects_dir}{cfg.subjectName}/ses{cfg.session-2}/recognition/../runRecording.csv")
    actualRuns_prepreDay = list(runRecording_prepreDay['run'].iloc[list(np.where(1==1*(runRecording_prepreDay['type']=='recognition'))[0])])[-(8-len(actualRuns)-len(actualRuns_preDay)):] # might be [5,6,7,8]
else:
    actualRuns_prepreDay = []

objects = ['bed', 'bench', 'chair', 'table']

for ii,run in enumerate(actualRuns): # load behavior and brain data for current session
    t = np.load(f"{cfg.recognition_dir}brain_run{run}.npy")
    mask = np.load(f"{cfg.chosenMask}")
    t = t[:,mask==1]
    t = normalize(t)
    brain_data=t if ii==0 else np.concatenate((brain_data,t), axis=0)

    t = pd.read_csv(f"{cfg.recognition_dir}behav_run{run}.csv")
    behav_data=t if ii==0 else pd.concat([behav_data,t])

for ii,run in enumerate(actualRuns_preDay): # load behavior and brain data for previous session
    t = np.load(f"{cfg.subjects_dir}{cfg.subjectName}/ses{cfg.session-1}/recognition/brain_run{run}.npy")
    mask = np.load(f"{cfg.chosenMask}")
    t = t[:,mask==1]
    t = normalize(t)
    brain_data = np.concatenate((brain_data,t), axis=0)

    t = pd.read_csv(f"{cfg.subjects_dir}{cfg.subjectName}/ses{cfg.session-1}/recognition/behav_run{run}.csv")
    behav_data = pd.concat([behav_data,t])

for ii,run in enumerate(actualRuns_prepreDay): # load behavior and brain data for previous session
    t = np.load(f"{cfg.subjects_dir}{cfg.subjectName}/ses{cfg.session-2}/recognition/brain_run{run}.npy")
    mask = np.load(f"{cfg.chosenMask}")
    t = t[:,mask==1]
    t = normalize(t)
    brain_data = np.concatenate((brain_data,t), axis=0)

    t = pd.read_csv(f"{cfg.subjects_dir}{cfg.subjectName}/ses{cfg.session-2}/recognition/behav_run{run}.csv")
    behav_data = pd.concat([behav_data,t])

# FEAT=brain_data.reshape(brain_data.shape[0],-1)

FEAT=brain_data
print(f"FEAT.shape={FEAT.shape}")
assert len(FEAT.shape)==2
# FEAT_mean=np.mean(FEAT,axis=1)
# FEAT=(FEAT.T-FEAT_mean).T
# FEAT_mean=np.mean(FEAT,axis=0)
# FEAT=FEAT-FEAT_mean
# FEAT = normalize(FEAT)

META=behav_data

# convert item colume to label colume
imcodeDict={
'A': 'bed',
'B': 'chair',
'C': 'table',
'D': 'bench'}
label=[]
for curr_trial in range(META.shape[0]):
    label.append(imcodeDict[META['Item'].iloc[curr_trial]])
META['label']=label # merge the label column with the data dataframe

# Which run to use as test data (leave as None to not have test data)
testRun = 2 # when testing: testRun = 2 ; META['run_num'].iloc[:5]=2

# Decide on the proportion of crescent data to use for classification
allpairs = itertools.combinations(objects,2)
accs={}
# Iterate over all the possible target pairs of objects
for pair in allpairs:
    # Find the control (remaining) objects for this pair
    altpair = other(pair)

    # pull sorted indices for each of the critical objects, in order of importance (low to high)
    # inds = get_inds(FEAT, META, pair, testRun=testRun)

    # Find the number of voxels that will be left given your inclusion parameter above
    # nvox = red_vox(FEAT.shape[1], include)

    for obj in pair:
        # foil = [i for i in pair if i != obj][0]
        for altobj in altpair:

            # establish a naming convention where it is $TARGET_$CLASSIFICATION
            # Target is the NF pair (e.g. bed/bench)
            # Classificationis is btw one of the targets, and a control (e.g. bed/chair, or bed/table, NOT bed/bench)
            naming = '{}{}_{}{}'.format(pair[0], pair[1], obj, altobj)

            # Pull the relevant inds from your previously established dictionary 
            # obj_inds = inds[obj]

            # If you're using testdata, this function will split it up. Otherwise it leaves out run as a parameter
            # if testRun:
            #     trainIX = META.index[(META['label'].isin([obj, altobj])) & (META['run_num'] != int(testRun))]
            #     testIX = META.index[(META['label'].isin([obj, altobj])) & (META['run_num'] == int(testRun))]
            # else:
            #     trainIX = META.index[(META['label'].isin([obj, altobj]))]
            #     testIX = META.index[(META['label'].isin([obj, altobj]))]
            # # pull training and test data
            # trainX = FEAT[trainIX]
            # testX = FEAT[testIX]
            # trainY = META.iloc[trainIX].label
            # testY = META.iloc[testIX].label

            # print(f"obj={obj},altobj={altobj}")
            # print(f"unique(trainY)={np.unique(trainY)}")
            # print(f"unique(testY)={np.unique(testY)}")
            # assert len(np.unique(trainY))==2

            if testRun:
                trainIX = ((META['label']==obj) + (META['label']==altobj)) * (META['run_num']!=int(testRun))
                testIX = ((META['label']==obj) + (META['label']==altobj)) * (META['run_num']==int(testRun))
            else:
                trainIX = ((META['label']==obj) + (META['label']==altobj))
                testIX = ((META['label']==obj) + (META['label']==altobj))
            # pull training and test data
            trainX = FEAT[trainIX]
            testX = FEAT[testIX]
            trainY = META.iloc[np.asarray(trainIX)].label
            testY = META.iloc[np.asarray(testIX)].label

            print(f"obj={obj},altobj={altobj}")
            print(f"unique(trainY)={np.unique(trainY)}")
            print(f"unique(testY)={np.unique(testY)}")
            assert len(np.unique(trainY))==2

            # # If you're selecting high-importance features, this bit handles that
            # if include < 1:
            #     trainX = trainX[:, obj_inds[-nvox:]]
            #     testX = testX[:, obj_inds[-nvox:]]

            # Train your classifier
            clf = LogisticRegression(penalty='l2',C=1, solver='lbfgs', max_iter=1000, 
                                     multi_class='multinomial').fit(trainX, trainY)

            model_folder = cfg.trainingModel_dir
            # Save it for later use
            joblib.dump(clf, model_folder +'/{}.joblib'.format(naming))

            # Monitor progress by printing accuracy (only useful if you're running a test set)
            acc = clf.score(testX, testY)
            print(naming, acc)
            accs[naming]=acc


def evidence(trainX,trainY):
    def classifierEvidence(clf,X,Y):
        ID=np.where((clf.classes_==Y[0])*1==1)[0][0]
        Evidence=(X@clf.coef_.T+clf.intercept_) if ID==1 else (-(X@clf.coef_.T+clf.intercept_))
        # Evidence=(X@clf.coef_.T+clf.intercept_) if ID==0 else (-(X@clf.coef_.T+clf.intercept_))
        return np.asarray(Evidence)
    FEAT=trainX
    META=trainY

    A_ID = (META['label']=='bed')
    X = FEAT[A_ID]

    print("floor")
    # D evidence for AD_clf when A is presented.
    Y = ['bench'] * X.shape[0]
    AD_clf=joblib.load(cfg.usingModel_dir +'bedchair_bedbench.joblib') # These 4 clf are the same:   bedchair_bedbench.joblib bedtable_bedbench.joblib benchchair_benchbed.joblib benchtable_benchbed.joblib
    AD_D_evidence = classifierEvidence(AD_clf,X,Y)
    evidence_floor = np.mean(AD_D_evidence)
    print(f"D evidence for AD_clf when A is presented={evidence_floor}")

    # C evidence for AC_clf when A is presented.
    Y = ['table'] * X.shape[0]
    AC_clf=joblib.load(cfg.usingModel_dir +'benchtable_tablebed.joblib') # These 4 clf are the same:   bedbench_bedtable.joblib bedchair_bedtable.joblib benchtable_tablebed.joblib chairtable_tablebed.joblib
    AC_C_evidence = classifierEvidence(AC_clf,X,Y)
    evidence_floor = np.mean(AC_C_evidence)
    print(f"C evidence for AC_clf when A is presented={evidence_floor}")


    # D evidence for CD_clf when A is presented.
    Y = ['bench'] * X.shape[0]
    CD_clf=joblib.load(cfg.usingModel_dir +'bedbench_benchtable.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
    CD_D_evidence = classifierEvidence(CD_clf,X,Y)
    evidence_floor = np.mean(CD_D_evidence)
    print(f"D evidence for CD_clf when A is presented={evidence_floor}")

    # C evidence for CD_clf when A is presented.
    Y = ['table'] * X.shape[0]
    CD_clf=joblib.load(cfg.usingModel_dir +'bedbench_benchtable.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
    CD_C_evidence = classifierEvidence(CD_clf,X,Y)
    evidence_floor = np.mean(CD_C_evidence)
    print(f"C evidence for CD_clf when A is presented={evidence_floor}")





    print("ceil")
    # evidence_ceil  is A evidence in AC and AD classifier
    Y = ['bed'] * X.shape[0]
    AC_clf=joblib.load(cfg.usingModel_dir +'benchtable_tablebed.joblib') # These 4 clf are the same:   bedbench_bedtable.joblib bedchair_bedtable.joblib benchtable_tablebed.joblib chairtable_tablebed.joblib
    AC_A_evidence = classifierEvidence(AC_clf,X,Y)
    evidence_ceil1 = AC_A_evidence
    print(f"A evidence in AC_clf when A is presented={np.mean(evidence_ceil1)}")

    Y = ['bed'] * X.shape[0]
    AD_clf=joblib.load(cfg.usingModel_dir +'bedchair_bedbench.joblib') # These 4 clf are the same:   bedchair_bedbench.joblib bedtable_bedbench.joblib benchchair_benchbed.joblib benchtable_benchbed.joblib
    AD_A_evidence = classifierEvidence(AD_clf,X,Y)
    evidence_ceil2 = AD_A_evidence
    print(f"A evidence in AD_clf when A is presented={np.mean(evidence_ceil2)}")

    # evidence_ceil = np.mean(evidence_ceil1)
    # evidence_ceil = np.mean(evidence_ceil2)
    evidence_ceil = np.mean((evidence_ceil1+evidence_ceil2)/2)
    print(f"evidence_ceil={evidence_ceil}")

# print the evidence using model training data

# print the evidence using model testing data

evidence(FEAT,META)


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/scipy/stats/stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in P

FEAT.shape=(384, 546)
obj=bed,altobj=chair
unique(trainY)=['bed' 'chair']
unique(testY)=['bed' 'chair']
bedbench_bedchair 0.6666666666666666
obj=bed,altobj=table
unique(trainY)=['bed' 'table']
unique(testY)=['bed' 'table']
bedbench_bedtable 0.5833333333333334
obj=bench,altobj=chair
unique(trainY)=['bench' 'chair']
unique(testY)=['bench' 'chair']
bedbench_benchchair 0.5416666666666666
obj=bench,altobj=table
unique(trainY)=['bench' 'table']
unique(testY)=['bench' 'table']


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/use

bedbench_benchtable 0.8333333333333334
obj=bed,altobj=bench
unique(trainY)=['bed' 'bench']
unique(testY)=['bed' 'bench']
bedchair_bedbench 0.75
obj=bed,altobj=table
unique(trainY)=['bed' 'table']
unique(testY)=['bed' 'table']
bedchair_bedtable 0.5833333333333334
obj=chair,altobj=bench
unique(trainY)=['bench' 'chair']
unique(testY)=['bench' 'chair']
bedchair_chairbench 0.5416666666666666
obj=chair,altobj=table
unique(trainY)=['chair' 'table']
unique(testY)=['chair' 'table']
bedchair_chairtable 0.75
obj=bed,altobj=bench
unique(trainY)=['bed' 'bench']
unique(testY)=['bed' 'bench']


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/use

...

benchchair_chairbed 0.6666666666666666
obj=chair,altobj=table
unique(trainY)=['chair' 'table']
unique(testY)=['chair' 'table']
benchchair_chairtable 0.75
obj=bench,altobj=bed
unique(trainY)=['bed' 'bench']
unique(testY)=['bed' 'bench']
benchtable_benchbed 0.75
obj=bench,altobj=chair
unique(trainY)=['bench' 'chair']
unique(testY)=['bench' 'chair']
benchtable_benchchair 0.5416666666666666
obj=table,altobj=bed
unique(trainY)=['bed' 'table']
unique(testY)=['bed' 'table']
benchtable_tablebed 0.5833333333333334
obj=table,altobj=chair
unique(trainY)=['chair' 'table']
unique(testY)=['chair' 'table']
benchtable_tablechair 0.75


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/use

obj=chair,altobj=bed
unique(trainY)=['bed' 'chair']
unique(testY)=['bed' 'chair']
chairtable_chairbed 0.6666666666666666
obj=chair,altobj=bench
unique(trainY)=['bench' 'chair']
unique(testY)=['bench' 'chair']
chairtable_chairbench 0.5416666666666666
obj=table,altobj=bed
unique(trainY)=['bed' 'table']
unique(testY)=['bed' 'table']
chairtable_tablebed 0.5833333333333334
obj=table,altobj=bench
unique(trainY)=['bench' 'table']
unique(testY)=['bench' 'table']
chairtable_tablebench 0.8333333333333334
trainX


/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/users/kp578/CONDA/rtcloud/lib/python3.6/site-packages/pandas/core/computation/expressions.py:201: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
/gpfs/milgram/project/turk-browne/use

KeyError: 'label'

In [149]:
FEAT,META=trainX,trainY
def classifierEvidence(clf,X,Y):
    ID=np.where((clf.classes_==Y[0])*1==1)[0][0]
    Evidence=(X@clf.coef_.T+clf.intercept_) if ID==1 else (-(X@clf.coef_.T+clf.intercept_))
    # Evidence=(X@clf.coef_.T+clf.intercept_) if ID==0 else (-(X@clf.coef_.T+clf.intercept_))
    return np.asarray(Evidence)
FEAT=trainX
META=trainY

A_ID = META=='bed'
X = FEAT[A_ID]

print("floor")
# D evidence for AD_clf when A is presented.
Y = ['bench'] * X.shape[0]
AD_clf=joblib.load(cfg.trainingModel_dir +'bedchair_bedbench.joblib') # These 4 clf are the same:   bedchair_bedbench.joblib bedtable_bedbench.joblib benchchair_benchbed.joblib benchtable_benchbed.joblib
AD_D_evidence = classifierEvidence(AD_clf,X,Y)
evidence_floor = np.mean(AD_D_evidence)
print(f"D evidence for AD_clf when A is presented={evidence_floor}")

# C evidence for AC_clf when A is presented.
Y = ['table'] * X.shape[0]
AC_clf=joblib.load(cfg.trainingModel_dir +'benchtable_tablebed.joblib') # These 4 clf are the same:   bedbench_bedtable.joblib bedchair_bedtable.joblib benchtable_tablebed.joblib chairtable_tablebed.joblib
AC_C_evidence = classifierEvidence(AC_clf,X,Y)
evidence_floor = np.mean(AC_C_evidence)
print(f"C evidence for AC_clf when A is presented={evidence_floor}")


# D evidence for CD_clf when A is presented.
Y = ['bench'] * X.shape[0]
CD_clf=joblib.load(cfg.trainingModel_dir +'bedbench_benchtable.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
CD_D_evidence = classifierEvidence(CD_clf,X,Y)
evidence_floor = np.mean(CD_D_evidence)
print(f"D evidence for CD_clf when A is presented={evidence_floor}")

# C evidence for CD_clf when A is presented.
Y = ['table'] * X.shape[0]
CD_clf=joblib.load(cfg.trainingModel_dir +'bedbench_benchtable.joblib') # These 4 clf are the same: bedbench_benchtable.joblib bedtable_tablebench.joblib benchchair_benchtable.joblib chairtable_tablebench.joblib
CD_C_evidence = classifierEvidence(CD_clf,X,Y)
evidence_floor = np.mean(CD_C_evidence)
print(f"C evidence for CD_clf when A is presented={evidence_floor}")





print("ceil")
# evidence_ceil  is A evidence in AC and AD classifier
Y = ['bed'] * X.shape[0]
AC_clf=joblib.load(cfg.trainingModel_dir +'benchtable_tablebed.joblib') # These 4 clf are the same:   bedbench_bedtable.joblib bedchair_bedtable.joblib benchtable_tablebed.joblib chairtable_tablebed.joblib
AC_A_evidence = classifierEvidence(AC_clf,X,Y)
evidence_ceil1 = AC_A_evidence
print(f"A evidence in AC_clf when A is presented={np.mean(evidence_ceil1)}")

Y = ['bed'] * X.shape[0]
AD_clf=joblib.load(cfg.trainingModel_dir +'bedchair_bedbench.joblib') # These 4 clf are the same:   bedchair_bedbench.joblib bedtable_bedbench.joblib benchchair_benchbed.joblib benchtable_benchbed.joblib
AD_A_evidence = classifierEvidence(AD_clf,X,Y)
evidence_ceil2 = AD_A_evidence
print(f"A evidence in AD_clf when A is presented={np.mean(evidence_ceil2)}")

# evidence_ceil = np.mean(evidence_ceil1)
# evidence_ceil = np.mean(evidence_ceil2)
evidence_ceil = np.mean((evidence_ceil1+evidence_ceil2)/2)
print(f"evidence_ceil={evidence_ceil}")


floor


IndexError: list index out of range

In [151]:
Y

[]